# Module 1 - Cross Validation

Datasets:

1. [Credit Card Fraud Detection](https://www.kaggle.com/dalpozz/creditcardfraud)

Dataset download:

1. https://www.kaggle.com/dalpozz/creditcardfraud/downloads/creditcardfraud.zip

References:

+ [Cross-validation @ scikit-learn](http://scikit-learn.org/stable/modules/cross_validation.html)
+ https://www.kaggle.com/arathee2/achieving-100-accuracy
+ https://www.kaggle.com/edunuke/binary-decision-tree-with-cross-validation/notebook
+ [Why every statistician should know about cross-validation](https://robjhyndman.com/hyndsight/crossvalidation/)

More:

+ https://www.kaggle.com/maximilianhahn/manager-skill-for-cross-validation-pipelines
+ https://www.kaggle.com/alexandrebarachant/simple-grasp-cross-validation/code
+ https://www.kaggle.com/solomonk/proper-cross-validation

In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.base import BaseEstimator
from sklearn.base import TransformerMixin

DATASET = lambda fname: os.path.join('datasets', fname)

## Load dataset

Check dataset dimension and class distribution.

In [2]:
assert os.path.exists(DATASET('creditcard.csv'))
dataset_original = pd.read_csv(DATASET('creditcard.csv'))

def brief_dataset(ds):
    """ Print a brief report of the dataset """
    print('shape', ds.shape,
          'distribution', {cls:ds.loc[lambda i: i.Class == cls, :].shape for cls in ds.Class.unique()})
    
brief_dataset(dataset_original)

shape (284807, 31) distribution {0: (284315, 31), 1: (492, 31)}


Take first a few rows for a preview of the dataset.

In [3]:
dataset_original[:5]

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


## Resample

Resample dataset to get more balanced class distribution.

API ref: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sample.html

In [4]:
subset_normal = dataset_original.loc[lambda i: i.Class == 0, :].sample(n = 4000) # downsample
subset_fraud = dataset_original.loc[lambda i: i.Class == 1, :].sample(n = 4000, replace = True) # upsample
dataset_resampled = pd.concat([subset_normal, subset_fraud]).reset_index(drop=True)
brief_dataset(dataset_resampled)

shape (8000, 31) distribution {0: (4000, 31), 1: (4000, 31)}


# SVM Training

API ref:

+ [Indexing @ pandas](http://pandas.pydata.org/pandas-docs/stable/indexing.html#different-choices-for-indexing)
+ [SVM API @ scikit-learn](http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html)

In [5]:
import sklearn.model_selection
from sklearn import svm

X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(
    dataset_resampled.iloc[:,:-1], dataset_resampled.Class, test_size=0.3)

print('shapes:', [i.shape for i in (X_train, X_test, y_train, y_test)])

classifier = svm.SVC(kernel='rbf', C=1, gamma=0.1).fit(X_train, y_train)
classifier.score(X_test, y_test)

shapes: [(5600, 30), (2400, 30), (5600,), (2400,)]


0.99916666666666665

# Computing cross-validated metrics

Create another resample for cross-validation. (Only because 280000+ rows takes too long)

In [6]:
dataset_cv = dataset_original.sample(n = 5000).reset_index(drop=True)
sklearn.model_selection.cross_val_score(classifier, dataset_cv.iloc[:,:-1], dataset_cv.Class, cv=10)

array([ 0.99600798,  0.99600798,  0.99600798,  0.99600798,  0.998     ,
        0.998     ,  0.99799599,  0.99799599,  0.99799599,  0.99799599])